In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import folium
import requests
import json

In [2]:
df = pd.read_csv('datos/SESNSP/IDEFC_NM_ene24.csv', encoding = "ISO-8859-1")
df = df.fillna(value = 0)

In [3]:
df['Enero'] = df['Enero'].astype(float)

In [4]:
df = df.groupby(['Año', 'Clave_Ent', 'Entidad','Tipo de delito']).sum().reset_index()
df.drop(columns = ['Bien jurídico afectado','Subtipo de delito','Modalidad'], inplace = True)
df = df[df['Año'] < 2024]

In [5]:
df['Total_delitos'] = df.iloc[:, -12:].sum(axis=1)

In [6]:
df = df.drop(df.columns[4:16], axis=1)

In [7]:
df

,Año,Clave_Ent,Entidad,Tipo de delito,Total_delitos
0,2015,1,Aguascalientes,Aborto,5.0
1,2015,1,Aguascalientes,Abuso de confianza,419.0
2,2015,1,Aguascalientes,Abuso sexual,18.0
3,2015,1,Aguascalientes,Acoso sexual,0.0
4,2015,1,Aguascalientes,Allanamiento de morada,169.0
...,...,...,...,...,...
11515,2023,32,Zacatecas,Tráfico de menores,0.0
11516,2023,32,Zacatecas,Violación equiparada,129.0
11517,2023,32,Zacatecas,Violación simple,161.0
11518,2023,32,Zacatecas,Violencia de género en todas sus modalidades d...,0.0


# Anexar informacion adicional

In [8]:
df.groupby('Entidad')['Clave_Ent'].mean()

Entidad
Aguascalientes                      1.0
Baja California                     2.0
Baja California Sur                 3.0
Campeche                            4.0
Chiapas                             7.0
Chihuahua                           8.0
Ciudad de México                    9.0
Coahuila de Zaragoza                5.0
Colima                              6.0
Durango                            10.0
Guanajuato                         11.0
Guerrero                           12.0
Hidalgo                            13.0
Jalisco                            14.0
Michoacán de Ocampo                16.0
Morelos                            17.0
México                             15.0
Nayarit                            18.0
Nuevo León                         19.0
Oaxaca                             20.0
Puebla                             21.0
Querétaro                          22.0
Quintana Roo                       23.0
San Luis Potosí                    24.0
Sinaloa                         

In [9]:
df.sort_values('Entidad')['Entidad'].unique()

array(['Aguascalientes', 'Baja California', 'Baja California Sur',
       'Campeche', 'Chiapas', 'Chihuahua', 'Ciudad de México',
       'Coahuila de Zaragoza', 'Colima', 'Durango', 'Guanajuato',
       'Guerrero', 'Hidalgo', 'Jalisco', 'Michoacán de Ocampo', 'Morelos',
       'México', 'Nayarit', 'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro',
       'Quintana Roo', 'San Luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco',
       'Tamaulipas', 'Tlaxcala', 'Veracruz de Ignacio de la Llave',
       'Yucatán', 'Zacatecas'], dtype=object)

### Agregando datos de poblacion

In [10]:
df_poblacion = pd.read_csv('datos/INEGI/Poblacion_01.csv')

In [11]:
df_poblacion

,Entidad federativa,2010,2020
0,Aguascalientes,1184996,1425607
1,Baja California,3155070,3769020
2,Baja California Sur,637026,798447
3,Campeche,822441,928363
4,Coahuila de Zaragoza,2748391,3146771
5,Colima,650555,731391
6,Chiapas,4796580,5543828
7,Chihuahua,3406465,3741869
8,Ciudad de México,8851080,9209944
9,Durango,1632934,1832650


In [12]:
#Realizar una interpolacion lineal para estimar la poblacion en los años faltantes
rango= [num for num in range(2011, 2024) if num != 2020]

for i in rango:
    df_poblacion[str(i)] = ((df_poblacion['2020']-df_poblacion['2010'])/10)*(i-2010) + df_poblacion['2010']

In [13]:
df_poblacion.sort_values('Entidad federativa', inplace=True)
df_poblacion.insert(1, 'Clave_Ent', list(range(1, 33)))

In [14]:
df_poblacion

,Entidad federativa,Clave_Ent,2010,2020,2011,2012,2013,2014,2015,2016,2017,2018,2019,2021,2022,2023
0,Aguascalientes,1,1184996,1425607,1209057.1,1233118.2,1257179.3,1281240.4,1305301.5,1329362.6,1353423.7,1377484.8,1401545.9,1449668.1,1473729.2,1497790.3
1,Baja California,2,3155070,3769020,3216465.0,3277860.0,3339255.0,3400650.0,3462045.0,3523440.0,3584835.0,3646230.0,3707625.0,3830415.0,3891810.0,3953205.0
2,Baja California Sur,3,637026,798447,653168.1,669310.2,685452.3,701594.4,717736.5,733878.6,750020.7,766162.8,782304.9,814589.1,830731.2,846873.3
3,Campeche,4,822441,928363,833033.2,843625.4,854217.6,864809.8,875402.0,885994.2,896586.4,907178.6,917770.8,938955.2,949547.4,960139.6
6,Chiapas,5,4796580,5543828,4871304.8,4946029.6,5020754.4,5095479.2,5170204.0,5244928.8,5319653.6,5394378.4,5469103.2,5618552.8,5693277.6,5768002.4
7,Chihuahua,6,3406465,3741869,3440005.4,3473545.8,3507086.2,3540626.6,3574167.0,3607707.4,3641247.8,3674788.2,3708328.6,3775409.4,3808949.8,3842490.2
8,Ciudad de México,7,8851080,9209944,8886966.4,8922852.8,8958739.2,8994625.6,9030512.0,9066398.4,9102284.8,9138171.2,9174057.6,9245830.4,9281716.8,9317603.2
4,Coahuila de Zaragoza,8,2748391,3146771,2788229.0,2828067.0,2867905.0,2907743.0,2947581.0,2987419.0,3027257.0,3067095.0,3106933.0,3186609.0,3226447.0,3266285.0
5,Colima,9,650555,731391,658638.6,666722.2,674805.8,682889.4,690973.0,699056.6,707140.2,715223.8,723307.4,739474.6,747558.2,755641.8
9,Durango,10,1632934,1832650,1652905.6,1672877.2,1692848.8,1712820.4,1732792.0,1752763.6,1772735.2,1792706.8,1812678.4,1852621.6,1872593.2,1892564.8


In [15]:
df_poblacion = df_poblacion.loc[:,'2010':].stack()

In [16]:
df_poblacion = df_poblacion.reset_index()
df_poblacion = df_poblacion.rename(columns={'level_0': 'Clave_Ent', 'level_1': 'Año', 0: 'Poblacion'})
df_poblacion['Clave_Ent'] += 1

In [17]:
df_poblacion

,Clave_Ent,Año,Poblacion
0,1,2010,1184996.0
1,1,2020,1425607.0
2,1,2011,1209057.1
3,1,2012,1233118.2
4,1,2013,1257179.3
...,...,...,...
443,32,2018,1595844.0
444,32,2019,1608991.0
445,32,2021,1635285.0
446,32,2022,1648432.0


In [18]:
df_poblacion['Año'] = df_poblacion['Año'].astype(int)

In [19]:
df_poblacion.dtypes

Clave_Ent      int64
Año            int32
Poblacion    float64
dtype: object

In [20]:
df = pd.merge(df, df_poblacion, on=['Clave_Ent', 'Año'], how='inner')

In [21]:
df

,Año,Clave_Ent,Entidad,Tipo de delito,Total_delitos,Poblacion
0,2015,1,Aguascalientes,Aborto,5.0,1305301.5
1,2015,1,Aguascalientes,Abuso de confianza,419.0,1305301.5
2,2015,1,Aguascalientes,Abuso sexual,18.0,1305301.5
3,2015,1,Aguascalientes,Acoso sexual,0.0,1305301.5
4,2015,1,Aguascalientes,Allanamiento de morada,169.0,1305301.5
...,...,...,...,...,...,...
11515,2023,32,Zacatecas,Tráfico de menores,0.0,1661579.0
11516,2023,32,Zacatecas,Violación equiparada,129.0,1661579.0
11517,2023,32,Zacatecas,Violación simple,161.0,1661579.0
11518,2023,32,Zacatecas,Violencia de género en todas sus modalidades d...,0.0,1661579.0


### Agregando datos de PIB

In [22]:
df_pib = pd.read_csv('datos/INEGI/PIBE_r_1.csv')
df_pib['2023'] = df_pib.iloc[:, -8:].mean(axis=1)
df_pib = df_pib.drop(df_pib.loc[:,'1980':'2014'], axis=1)

In [23]:
df_pib.sort_values('Concepto', inplace=True)
df_pib.insert(1, 'Clave_Ent', list(range(1, 33)))

In [24]:
df_pib

,Concepto,Clave_Ent,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Aguascalientes,1,294497,307396,314242,327926,327058,303556,312954,316500,313016.125
1,Baja California,2,795081,828692,863601,878817,897460,833361,901664,929459,866016.875
2,Baja California Sur,3,156072,163095,170784,184618,179624,152597,170695,177421,169363.250
3,Campeche,4,700311,663239,595667,564591,544914,502567,491694,473101,567010.500
6,Chiapas,5,364001,369919,359734,353600,349375,334057,349531,368789,356125.750
7,Chihuahua,6,803883,830489,851284,853323,862012,809562,849224,919617,847424.250
8,Ciudad de México,7,3503520,3571368,3646318,3694575,3688277,3292782,3480722,3640388,3564743.750
4,Coahuila de Zaragoza,8,897702,901719,941582,966055,974554,862347,916340,922287,922823.250
5,Colima,9,141029,140705,150360,153808,161221,148214,147102,148130,148821.125
9,Durango,10,300289,305382,304647,308981,308659,283133,299795,296963,300981.125


In [25]:
df_pib = df_pib.loc[:,'2015':].stack()

In [26]:
df_pib = df_pib.reset_index()
df_pib = df_pib.rename(columns={'level_0': 'Clave_Ent', 'level_1': 'Año', 0: 'PIB'})
df_pib['Clave_Ent'] += 1

In [27]:
df_pib

,Clave_Ent,Año,PIB
0,1,2015,294497.0
1,1,2016,307396.0
2,1,2017,314242.0
3,1,2018,327926.0
4,1,2019,327058.0
...,...,...,...
283,32,2019,219945.0
284,32,2020,210123.0
285,32,2021,219774.0
286,32,2022,221026.0


In [28]:
df_pib['Año'] = df_pib['Año'].astype(int)

In [29]:
df = pd.merge(df, df_pib, on=['Clave_Ent', 'Año'], how='inner')

In [30]:
df

,Año,Clave_Ent,Entidad,Tipo de delito,Total_delitos,Poblacion,PIB
0,2015,1,Aguascalientes,Aborto,5.0,1305301.5,294497.0
1,2015,1,Aguascalientes,Abuso de confianza,419.0,1305301.5,294497.0
2,2015,1,Aguascalientes,Abuso sexual,18.0,1305301.5,294497.0
3,2015,1,Aguascalientes,Acoso sexual,0.0,1305301.5,294497.0
4,2015,1,Aguascalientes,Allanamiento de morada,169.0,1305301.5,294497.0
...,...,...,...,...,...,...,...
11515,2023,32,Zacatecas,Tráfico de menores,0.0,1661579.0,218885.5
11516,2023,32,Zacatecas,Violación equiparada,129.0,1661579.0,218885.5
11517,2023,32,Zacatecas,Violación simple,161.0,1661579.0,218885.5
11518,2023,32,Zacatecas,Violencia de género en todas sus modalidades d...,0.0,1661579.0,218885.5


### Agregando datos de tasa de desocupacion

In [31]:
df_desoc = pd.read_csv('datos/INEGi/TasaDesocupacion.CSV')
df_desoc = df_desoc[df_desoc['Indicador'] == 'Total']
df_desoc = df_desoc.drop('Indicador', axis=1)
df_desoc = df_desoc.reset_index().drop('index', axis=1)

In [32]:
df_desoc.loc[:,'2019/01':]

,2019/01,2019/02,2019/03,2019/04,2020/01,2020/02,2020/03,2020/04,2021/01,2021/02,2021/03,2021/04,2022/01,2022/02,2022/03,2022/04,2023/01,2023/02,2023/03,2023/04
0,3.28535,3.67106,3.79282,3.34141,3.35009,NaN,5.19230,5.19754,3.62242,3.87156,4.62574,4.37726,3.51969,3.76820,4.03923,3.76756,3.01962,3.19796,3.17738,3.50581
1,2.51192,2.55760,2.89908,2.21851,2.16798,NaN,2.74817,2.88007,2.55476,2.44667,2.87954,2.04351,1.91049,2.50984,2.59254,2.70224,1.71222,2.30743,2.43889,2.24539
2,3.95454,4.61803,4.94592,4.37215,3.83703,NaN,7.38778,5.42497,4.98176,4.40950,4.25482,2.59220,2.87475,2.73295,2.86790,2.85079,2.75936,2.70070,3.18616,2.26334
3,3.47572,3.47649,3.00970,3.22312,2.78253,NaN,3.82608,3.38969,2.92643,2.89527,3.05700,2.95580,3.06809,2.95330,2.29772,1.97641,1.76240,1.75905,1.81924,1.71511
4,4.24849,4.88419,4.81236,4.27631,4.76719,NaN,7.37167,6.15794,5.11096,4.84208,4.93013,4.80892,5.02271,4.02557,4.24342,3.41466,3.62833,3.70273,3.87742,4.33968
5,3.91841,3.62257,3.81958,2.95786,3.12872,NaN,4.14453,3.80916,3.20564,3.19790,2.91652,3.12393,2.77123,2.56226,2.60985,2.35763,2.13478,2.20655,2.55965,2.30336
6,3.09453,3.60047,3.70345,2.45755,2.58386,NaN,3.39902,2.72578,3.17416,2.90871,3.28256,3.33131,2.49952,2.58906,2.28847,2.32698,2.48801,1.83913,2.47877,1.73476
7,3.03526,3.38451,3.19745,2.57083,3.49114,NaN,4.69780,3.73507,3.97549,3.61242,3.02856,2.86673,2.59984,2.45553,2.46536,1.98906,2.78441,2.47852,2.81498,1.92983
8,4.73037,5.15945,4.99577,5.10120,4.72524,NaN,7.99090,6.63142,7.30910,7.25668,6.99109,5.89173,5.84765,5.39836,5.25200,4.53060,3.75255,4.36955,4.06650,3.90904
9,4.64486,4.25457,3.94023,3.25998,4.05512,NaN,4.80873,4.87685,4.14994,4.59470,4.21080,3.43068,3.57521,3.26514,3.25675,3.08720,2.98737,2.87201,3.09686,2.82041


In [33]:
df_desoc = df_desoc.fillna(method = 'ffill', axis=1)

In [34]:
df_desoc.loc[:,'2019/01':]

,2019/01,2019/02,2019/03,2019/04,2020/01,2020/02,2020/03,2020/04,2021/01,2021/02,2021/03,2021/04,2022/01,2022/02,2022/03,2022/04,2023/01,2023/02,2023/03,2023/04
0,3.28535,3.67106,3.79282,3.34141,3.35009,3.35009,5.1923,5.19754,3.62242,3.87156,4.62574,4.37726,3.51969,3.7682,4.03923,3.76756,3.01962,3.19796,3.17738,3.50581
1,2.51192,2.5576,2.89908,2.21851,2.16798,2.16798,2.74817,2.88007,2.55476,2.44667,2.87954,2.04351,1.91049,2.50984,2.59254,2.70224,1.71222,2.30743,2.43889,2.24539
2,3.95454,4.61803,4.94592,4.37215,3.83703,3.83703,7.38778,5.42497,4.98176,4.4095,4.25482,2.5922,2.87475,2.73295,2.8679,2.85079,2.75936,2.7007,3.18616,2.26334
3,3.47572,3.47649,3.0097,3.22312,2.78253,2.78253,3.82608,3.38969,2.92643,2.89527,3.057,2.9558,3.06809,2.9533,2.29772,1.97641,1.7624,1.75905,1.81924,1.71511
4,4.24849,4.88419,4.81236,4.27631,4.76719,4.76719,7.37167,6.15794,5.11096,4.84208,4.93013,4.80892,5.02271,4.02557,4.24342,3.41466,3.62833,3.70273,3.87742,4.33968
5,3.91841,3.62257,3.81958,2.95786,3.12872,3.12872,4.14453,3.80916,3.20564,3.1979,2.91652,3.12393,2.77123,2.56226,2.60985,2.35763,2.13478,2.20655,2.55965,2.30336
6,3.09453,3.60047,3.70345,2.45755,2.58386,2.58386,3.39902,2.72578,3.17416,2.90871,3.28256,3.33131,2.49952,2.58906,2.28847,2.32698,2.48801,1.83913,2.47877,1.73476
7,3.03526,3.38451,3.19745,2.57083,3.49114,3.49114,4.6978,3.73507,3.97549,3.61242,3.02856,2.86673,2.59984,2.45553,2.46536,1.98906,2.78441,2.47852,2.81498,1.92983
8,4.73037,5.15945,4.99577,5.1012,4.72524,4.72524,7.9909,6.63142,7.3091,7.25668,6.99109,5.89173,5.84765,5.39836,5.252,4.5306,3.75255,4.36955,4.0665,3.90904
9,4.64486,4.25457,3.94023,3.25998,4.05512,4.05512,4.80873,4.87685,4.14994,4.5947,4.2108,3.43068,3.57521,3.26514,3.25675,3.0872,2.98737,2.87201,3.09686,2.82041


In [35]:
for i in range(1,10):
    year = 2024-i
    df_desoc.insert(1, str(year), df_desoc.iloc[:,-4:].mean(axis=1))
    df_desoc.drop(df_desoc.iloc[:,-4:], axis=1, inplace=True)

In [36]:
df_desoc.sort_values('Entidad federativa', inplace=True)
df_desoc.insert(1, 'Clave_Ent', list(range(1, 33)))

In [37]:
df_desoc

,Entidad federativa,Clave_Ent,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Aguascalientes,1,4.503325,3.972855,3.630322,3.4501,3.52266,4.272505,4.124245,3.77367,3.225192
1,Baja California,2,4.16816,2.688038,2.852475,2.51368,2.546778,2.49105,2.48112,2.428777,2.175982
2,Baja California Sur,3,4.786112,4.60931,4.35683,3.782522,4.47266,5.121702,4.05957,2.831597,2.72739
3,Campeche,4,2.884725,3.702132,3.803275,3.26537,3.296257,3.195207,2.958625,2.57388,1.76395
6,Chiapas,5,3.093762,3.062107,2.574955,2.78294,3.214,2.82313,3.174185,2.426007,2.135168
7,Chihuahua,6,3.728603,3.158615,2.70045,3.153355,3.047013,3.853788,3.3708,2.377448,2.501935
8,Ciudad de México,7,5.344278,5.03398,4.602907,4.61948,4.996697,6.0182,6.86215,5.257153,4.02441
4,Coahuila de Zaragoza,8,5.101025,4.449495,4.388165,4.14186,4.555338,5.765998,4.923023,4.17659,3.88704
5,Colima,9,4.677623,4.083425,3.62567,3.252147,3.579605,3.552782,3.110997,2.575242,2.301085
9,Durango,10,4.901497,4.492987,3.818468,4.290887,4.02491,4.448955,4.09653,3.296075,2.944163


In [38]:
df_desoc = df_desoc.loc[:,'2015':].stack()

In [39]:
df_desoc = df_desoc.reset_index()
df_desoc = df_desoc.rename(columns={'level_0': 'Clave_Ent', 'level_1': 'Año', 0: 'Desocupacion'})
df_desoc['Clave_Ent'] += 1

In [40]:
df_desoc

,Clave_Ent,Año,Desocupacion
0,1,2015,4.503325
1,1,2016,3.972855
2,1,2017,3.630322
3,1,2018,3.4501
4,1,2019,3.52266
...,...,...,...
283,32,2019,3.061935
284,32,2020,3.67699
285,32,2021,3.504418
286,32,2022,2.797048


In [41]:
df_desoc['Año'] = df_desoc['Año'].astype(int)

In [42]:
df = pd.merge(df, df_desoc, on=['Clave_Ent', 'Año'], how='inner')

In [43]:
df

,Año,Clave_Ent,Entidad,Tipo de delito,Total_delitos,Poblacion,PIB,Desocupacion
0,2015,1,Aguascalientes,Aborto,5.0,1305301.5,294497.0,4.503325
1,2015,1,Aguascalientes,Abuso de confianza,419.0,1305301.5,294497.0,4.503325
2,2015,1,Aguascalientes,Abuso sexual,18.0,1305301.5,294497.0,4.503325
3,2015,1,Aguascalientes,Acoso sexual,0.0,1305301.5,294497.0,4.503325
4,2015,1,Aguascalientes,Allanamiento de morada,169.0,1305301.5,294497.0,4.503325
...,...,...,...,...,...,...,...,...
11515,2023,32,Zacatecas,Tráfico de menores,0.0,1661579.0,218885.5,2.872008
11516,2023,32,Zacatecas,Violación equiparada,129.0,1661579.0,218885.5,2.872008
11517,2023,32,Zacatecas,Violación simple,161.0,1661579.0,218885.5,2.872008
11518,2023,32,Zacatecas,Violencia de género en todas sus modalidades d...,0.0,1661579.0,218885.5,2.872008


### Agregando datos de Eficiencia terminal de la enseñanza primaria

In [44]:
df_efic = pd.read_csv('datos/INEGI/EficienciaTerminal.csv')

In [45]:
df_efic

,Entidad federativa,2015,2016,2017,2018,2019,2020,2021
0,Aguascalientes,99.48262,99.35626,100.07392,97.48356,98.09545,100.23181,98.33755
1,Baja California,98.14218,98.66109,97.24271,98.01292,98.76864,100.17642,99.27973
2,Baja California Sur,100.85804,100.22020,101.06473,101.19988,101.44480,100.89693,100.91770
3,Campeche,97.12600,95.57463,95.45344,92.90948,92.72780,94.99509,94.89013
4,Coahuila de Zaragoza,98.00489,98.33659,99.06695,97.32727,97.75960,98.07359,99.20941
5,Colima,104.72692,98.76622,97.61600,91.93321,93.91699,95.10276,92.44448
6,Chiapas,95.16524,94.79580,94.79650,92.83203,92.61559,94.33084,97.04795
7,Chihuahua,97.12574,99.88062,97.52694,97.61851,99.30733,100.06265,98.30975
8,Ciudad de México,98.97006,96.48690,97.40903,94.05605,93.73268,94.42753,90.76606
9,Durango,98.18177,96.03335,95.33467,95.17021,95.17954,96.66245,94.46514


In [46]:
df_efic['2022'] = df_efic.iloc[:,-7:].mean(axis=1)
df_efic['2023'] = df_efic.iloc[:,-7:].mean(axis=1)

In [47]:
df_efic.sort_values('Entidad federativa', inplace=True)
df_efic.insert(1, 'Clave_Ent', list(range(1, 33)))

In [48]:
df_efic

,Entidad federativa,Clave_Ent,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Aguascalientes,1,99.48262,99.35626,100.07392,97.48356,98.09545,100.23181,98.33755,99.008739,98.941041
1,Baja California,2,98.14218,98.66109,97.24271,98.01292,98.76864,100.17642,99.27973,98.611956,98.679067
2,Baja California Sur,3,100.85804,100.22020,101.06473,101.19988,101.44480,100.89693,100.91770,100.943183,100.955346
3,Campeche,4,97.12600,95.57463,95.45344,92.90948,92.72780,94.99509,94.89013,94.810939,94.480216
6,Chiapas,5,95.16524,94.79580,94.79650,92.83203,92.61559,94.33084,97.04795,94.511993,94.418672
7,Chihuahua,6,97.12574,99.88062,97.52694,97.61851,99.30733,100.06265,98.30975,98.547363,98.750452
8,Ciudad de México,7,98.97006,96.48690,97.40903,94.05605,93.73268,94.42753,90.76606,95.121187,94.571348
4,Coahuila de Zaragoza,8,98.00489,98.33659,99.06695,97.32727,97.75960,98.07359,99.20941,98.254043,98.289636
5,Colima,9,104.72692,98.76622,97.61600,91.93321,93.91699,95.10276,92.44448,96.358083,95.162535
9,Durango,10,98.18177,96.03335,95.33467,95.17021,95.17954,96.66245,94.46514,95.861019,95.529483


In [49]:
df_efic = df_efic.loc[:,'2015':].stack()

In [50]:
df_efic = df_efic.reset_index()
df_efic = df_efic.rename(columns={'level_0': 'Clave_Ent', 'level_1': 'Año', 0: 'EficienciaPrimaria'})
df_efic['Clave_Ent'] += 1

In [51]:
df_efic

,Clave_Ent,Año,EficienciaPrimaria
0,1,2015,99.482620
1,1,2016,99.356260
2,1,2017,100.073920
3,1,2018,97.483560
4,1,2019,98.095450
...,...,...,...
283,32,2019,96.272020
284,32,2020,97.479670
285,32,2021,96.752700
286,32,2022,96.658114


In [52]:
df_efic['Año'] = df_efic['Año'].astype(int)

In [53]:
df = pd.merge(df, df_efic, on=['Clave_Ent', 'Año'], how='inner')

In [54]:
df

,Año,Clave_Ent,Entidad,Tipo de delito,Total_delitos,Poblacion,PIB,Desocupacion,EficienciaPrimaria
0,2015,1,Aguascalientes,Aborto,5.0,1305301.5,294497.0,4.503325,99.482620
1,2015,1,Aguascalientes,Abuso de confianza,419.0,1305301.5,294497.0,4.503325,99.482620
2,2015,1,Aguascalientes,Abuso sexual,18.0,1305301.5,294497.0,4.503325,99.482620
3,2015,1,Aguascalientes,Acoso sexual,0.0,1305301.5,294497.0,4.503325,99.482620
4,2015,1,Aguascalientes,Allanamiento de morada,169.0,1305301.5,294497.0,4.503325,99.482620
...,...,...,...,...,...,...,...,...,...
11515,2023,32,Zacatecas,Tráfico de menores,0.0,1661579.0,218885.5,2.872008,96.367071
11516,2023,32,Zacatecas,Violación equiparada,129.0,1661579.0,218885.5,2.872008,96.367071
11517,2023,32,Zacatecas,Violación simple,161.0,1661579.0,218885.5,2.872008,96.367071
11518,2023,32,Zacatecas,Violencia de género en todas sus modalidades d...,0.0,1661579.0,218885.5,2.872008,96.367071


### Medicos por cada cien mil habitantes

In [55]:
df_med = pd.read_csv('datos/INEGI/Medicos.csv')

In [56]:
df_med['2023'] = df_med.iloc[:,-7:].mean(axis=1)

In [57]:
df_med.sort_values('Entidad federativa', inplace=True)
df_med.insert(1, 'Clave_Ent', list(range(1, 33)))

In [58]:
df_med

,Entidad federativa,Clave_Ent,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Aguascalientes,1,1.90087,1.98550,2.16623,2.06086,2.15200,2.13729,2.20148,2.23791,2.134467
1,Baja California,2,1.58419,1.59542,1.58135,1.49628,1.52373,1.55889,1.54347,1.48371,1.540407
2,Baja California Sur,3,2.33295,2.43296,2.36968,2.36669,2.44686,2.39971,2.41515,2.37040,2.400207
3,Campeche,4,2.05327,2.15508,2.14011,2.49056,2.40656,2.42233,2.33093,2.38935,2.333560
6,Chiapas,5,1.26116,1.36668,1.43907,1.40410,1.31228,1.35059,1.37342,1.41995,1.380870
7,Chihuahua,6,1.56480,1.64225,1.60839,1.73341,1.70000,1.66788,1.86558,1.84979,1.723900
8,Ciudad de México,7,3.79995,3.82686,3.87974,3.82702,3.86971,3.88584,4.01439,4.19122,3.927826
4,Coahuila de Zaragoza,8,1.80359,1.79146,1.82280,1.85342,1.91401,1.87481,1.92183,1.93606,1.873484
5,Colima,9,2.30107,2.46245,2.52845,2.63593,2.67699,2.70962,2.66733,2.75043,2.633029
9,Durango,10,1.95581,1.99922,1.99266,2.03489,2.01319,2.02728,2.21261,2.15307,2.061846


In [59]:
df_med = df_med.loc[:,'2015':].stack()

In [60]:
df_med = df_med.reset_index()
df_med = df_med.rename(columns={'level_0': 'Clave_Ent', 'level_1': 'Año', 0: 'Medicos'})
df_med['Clave_Ent'] += 1

In [61]:
df_med

,Clave_Ent,Año,Medicos
0,1,2015,1.900870
1,1,2016,1.985500
2,1,2017,2.166230
3,1,2018,2.060860
4,1,2019,2.152000
...,...,...,...
283,32,2019,1.921370
284,32,2020,1.917990
285,32,2021,2.000120
286,32,2022,2.011310


In [62]:
df_med['Año'] = df_med['Año'].astype(int)

In [63]:
df = pd.merge(df, df_med, on=['Clave_Ent', 'Año'], how='inner')

In [64]:
df

,Año,Clave_Ent,Entidad,Tipo de delito,Total_delitos,Poblacion,PIB,Desocupacion,EficienciaPrimaria,Medicos
0,2015,1,Aguascalientes,Aborto,5.0,1305301.5,294497.0,4.503325,99.482620,1.900870
1,2015,1,Aguascalientes,Abuso de confianza,419.0,1305301.5,294497.0,4.503325,99.482620,1.900870
2,2015,1,Aguascalientes,Abuso sexual,18.0,1305301.5,294497.0,4.503325,99.482620,1.900870
3,2015,1,Aguascalientes,Acoso sexual,0.0,1305301.5,294497.0,4.503325,99.482620,1.900870
4,2015,1,Aguascalientes,Allanamiento de morada,169.0,1305301.5,294497.0,4.503325,99.482620,1.900870
...,...,...,...,...,...,...,...,...,...,...
11515,2023,32,Zacatecas,Tráfico de menores,0.0,1661579.0,218885.5,2.872008,96.367071,1.932931
11516,2023,32,Zacatecas,Violación equiparada,129.0,1661579.0,218885.5,2.872008,96.367071,1.932931
11517,2023,32,Zacatecas,Violación simple,161.0,1661579.0,218885.5,2.872008,96.367071,1.932931
11518,2023,32,Zacatecas,Violencia de género en todas sus modalidades d...,0.0,1661579.0,218885.5,2.872008,96.367071,1.932931


# Aplicando algunos modelos de ML

## Solo homicidios

In [65]:
df_homicidios = df[df['Tipo de delito'] == 'Homicidio']

In [66]:
df_homicidios

,Año,Clave_Ent,Entidad,Tipo de delito,Total_delitos,Poblacion,PIB,Desocupacion,EficienciaPrimaria,Medicos
18,2015,1,Aguascalientes,Homicidio,211.0,1305301.5,294497.000,4.503325,99.482620,1.900870
58,2015,2,Baja California,Homicidio,1219.0,3462045.0,795081.000,4.16816,98.142180,1.584190
98,2015,3,Baja California Sur,Homicidio,201.0,717736.5,156072.000,4.786112,100.858040,2.332950
138,2015,4,Campeche,Homicidio,118.0,875402.0,700311.000,2.884725,97.126000,2.053270
178,2015,5,Coahuila de Zaragoza,Homicidio,535.0,2947581.0,897702.000,5.101025,98.004890,1.803590
...,...,...,...,...,...,...,...,...,...,...
11338,2023,28,Tamaulipas,Homicidio,1112.0,3605489.3,722614.000,3.124213,95.281778,2.016107
11378,2023,29,Tlaxcala,Homicidio,141.0,1394889.3,141448.750,3.335152,100.553584,1.765706
11418,2023,30,Veracruz de Ignacio de la Llave,Homicidio,1814.0,8188394.5,1035069.875,2.108605,94.883716,1.621420
11458,2023,31,Yucatán,Homicidio,165.0,2430494.3,350027.375,1.848492,98.527005,2.253207


In [67]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [68]:
X = df_homicidios.drop(['Clave_Ent', 'Año', 'Entidad','Tipo de delito', 'Total_delitos'], axis = 1)

In [69]:
y = df_homicidios['Total_delitos']

In [70]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size = 0.8)

In [71]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((230, 5), (58, 5), (230,), (58,))

### Regresion Lineal

In [72]:
from sklearn.linear_model import LinearRegression
LinearRegression = LinearRegression()

In [73]:
model_linear_regression = Pipeline([
    ('scaler', StandardScaler()),   # Step 1: Scale the features
    ('regression', LinearRegression)  # Step 2: Fit linear regression model
])

In [74]:
model_linear_regression.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('regression', LinearRegression())])

In [75]:
muestra_datos = X_test.iloc[:5]

model_linear_regression.predict(muestra_datos)

array([1832.05013721, 2061.76809698, 1004.44142727,  821.87887907,
        592.8332687 ])

In [76]:
list(y_test.iloc[:5])

[1873.0, 1466.0, 1421.0, 1228.0, 1001.0]

In [77]:
from sklearn.model_selection import cross_val_score

linear_scores = cross_val_score(model_linear_regression, X, y,
                               scoring = 'neg_mean_squared_error', cv =5)

linear_scores

array([-357483.94134195, -391533.30829133, -589698.59748333,
       -360209.34249044, -294328.7603252 ])

In [78]:
linear_rmse_scores = np.sqrt(-linear_scores)
linear_rmse_scores

array([597.89960808, 625.72622471, 767.9183534 , 600.17442672,
       542.52074645])

In [79]:
def display_scores(scores):
    print('Scores:', scores)
    print('Mean:', scores.mean())
    print('Standard Dev:', scores.std())

In [80]:
display_scores(linear_rmse_scores)

Scores: [597.89960808 625.72622471 767.9183534  600.17442672 542.52074645]
Mean: 626.8478718711676
Standard Dev: 75.58131724863333


### DecisionTree

In [81]:
#Aplicar el modelo de arboles de decision con DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor

#1. Nombrar una variable para nuestro modelo
tree_reg = DecisionTreeRegressor(random_state=42)

In [82]:
#2. Definir el Pipeline : Transformación  + modelo
model_tree_reg = Pipeline(steps = [
    ('scaler', StandardScaler()),
    ('tree_reg', tree_reg)
])

In [83]:
#3. Ajustar el modelo
model_tree_reg.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('tree_reg', DecisionTreeRegressor(random_state=42))])

In [84]:
#Comparar con el modelo de regresion lineal para una muestra de datos
print(model_linear_regression.predict(muestra_datos))
print(model_tree_reg.predict(muestra_datos))

[1832.05013721 2061.76809698 1004.44142727  821.87887907  592.8332687 ]
[2061. 1816. 1968.  231.  997.]


In [85]:
list(y_test.iloc[:5])

[1873.0, 1466.0, 1421.0, 1228.0, 1001.0]

In [86]:
from sklearn.model_selection import cross_val_score

tree_reg_scores = cross_val_score(model_tree_reg, X, y,
                               scoring = 'neg_mean_squared_error', cv = 5)

tree_reg_scores

array([-424561.67241379, -206708.29310345, -427914.65517241,
       -164898.98245614, -264197.14035088])

In [87]:
tree_reg_scores = np.sqrt(-tree_reg_scores)
tree_reg_scores

array([651.58397188, 454.65183724, 654.15185941, 406.07755719,
       514.00110929])

In [88]:
display_scores(tree_reg_scores)

Scores: [651.58397188 454.65183724 654.15185941 406.07755719 514.00110929]
Mean: 536.093267002964
Standard Dev: 101.2924369013957


### RandomForest

In [89]:
from sklearn.ensemble import RandomForestRegressor

In [90]:
forest_reg = RandomForestRegressor(n_estimators = 10, random_state = 42)

In [91]:
model_forest_reg = Pipeline(steps = [
    ('scaler', StandardScaler()),
    ('forest_reg', forest_reg)
])

In [92]:
model_forest_reg.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('forest_reg',
                 RandomForestRegressor(n_estimators=10, random_state=42))])

In [93]:
#Comparar con el modelo de regresion lineal para una muestra de datos
print(model_linear_regression.predict(muestra_datos))
print(model_forest_reg.predict(muestra_datos))
print(model_tree_reg.predict(muestra_datos))

[1832.05013721 2061.76809698 1004.44142727  821.87887907  592.8332687 ]
[2054.2 1828.8 1500.6  747.2  961.6]
[2061. 1816. 1968.  231.  997.]


In [94]:
#Comparamos con los valores reales
list(y_test.iloc[:5])

[1873.0, 1466.0, 1421.0, 1228.0, 1001.0]

In [95]:
from sklearn.model_selection import cross_val_score

forest_reg_scores = cross_val_score(model_forest_reg, X, y,
                               scoring = 'neg_mean_squared_error', cv = 5)

forest_reg_scores

array([-446738.42465517, -150980.80396552, -271018.41      ,
       -126954.70631579, -131850.12982456])

In [96]:
forest_reg_scores = np.sqrt(-forest_reg_scores)
forest_reg_scores

array([668.38493748, 388.56248399, 520.59428541, 356.30703939,
       363.11173187])

In [97]:
display_scores(forest_reg_scores)

Scores: [668.38493748 388.56248399 520.59428541 356.30703939 363.11173187]
Mean: 459.3920956286655
Standard Dev: 120.2804947866077


## Utilizando todos los datos

In [98]:
df

,Año,Clave_Ent,Entidad,Tipo de delito,Total_delitos,Poblacion,PIB,Desocupacion,EficienciaPrimaria,Medicos
0,2015,1,Aguascalientes,Aborto,5.0,1305301.5,294497.0,4.503325,99.482620,1.900870
1,2015,1,Aguascalientes,Abuso de confianza,419.0,1305301.5,294497.0,4.503325,99.482620,1.900870
2,2015,1,Aguascalientes,Abuso sexual,18.0,1305301.5,294497.0,4.503325,99.482620,1.900870
3,2015,1,Aguascalientes,Acoso sexual,0.0,1305301.5,294497.0,4.503325,99.482620,1.900870
4,2015,1,Aguascalientes,Allanamiento de morada,169.0,1305301.5,294497.0,4.503325,99.482620,1.900870
...,...,...,...,...,...,...,...,...,...,...
11515,2023,32,Zacatecas,Tráfico de menores,0.0,1661579.0,218885.5,2.872008,96.367071,1.932931
11516,2023,32,Zacatecas,Violación equiparada,129.0,1661579.0,218885.5,2.872008,96.367071,1.932931
11517,2023,32,Zacatecas,Violación simple,161.0,1661579.0,218885.5,2.872008,96.367071,1.932931
11518,2023,32,Zacatecas,Violencia de género en todas sus modalidades d...,0.0,1661579.0,218885.5,2.872008,96.367071,1.932931


In [99]:
numeric_list = ['Poblacion', 'PIB','Desocupacion','EficienciaPrimaria', 'Medicos']
categorical_list = ['Tipo de delito']

In [100]:
numeric_transformer = Pipeline( steps = [ ( 'Scaler', StandardScaler() ) ] )
categorical_transformer = Pipeline( steps = [ ( 'onehot', OneHotEncoder() ) ] )

In [101]:
transformer = ColumnTransformer([
    ('num', numeric_transformer, numeric_list), ('cat', categorical_transformer, categorical_list)
])

In [102]:
X = df.drop(['Total_delitos', 'Año', 'Entidad','Clave_Ent'], axis = 1)

In [103]:
y = df['Total_delitos']

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size = 0.85)

In [105]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((9792, 6), (1728, 6), (9792,), (1728,))

### Regresion Lineal

In [106]:
from sklearn.linear_model import LinearRegression
linear_regression = LinearRegression()

In [107]:
model_linear_regression2 = Pipeline([
    ('tranformer', transformer), ('linear_regression', linear_regression)
])

In [108]:
model_linear_regression2.fit(X_train, y_train)

Pipeline(steps=[('tranformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('Scaler',
                                                                   StandardScaler())]),
                                                  ['Poblacion', 'PIB',
                                                   'Desocupacion',
                                                   'EficienciaPrimaria',
                                                   'Medicos']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  ['Tipo de delito'])])),
                ('linear_regression', LinearRegression())])

In [109]:
muestra_datos = X_test.iloc[:5]

model_linear_regression2.predict(muestra_datos)

array([ -318.12701476,    79.95188995, -1200.27987468,  4414.57842882,
       -1212.52053693])

In [110]:
list(y_test.iloc[:5])

[42.0, 829.0, 0.0, 532.0, 465.0]

In [111]:
from sklearn.model_selection import cross_val_score

linear_scores = cross_val_score(model_linear_regression2, X, y,
                               scoring = 'neg_mean_squared_error', cv =5)

linear_scores

array([-28952907.77195829, -37748214.81112184, -21386823.16564592,
       -25016421.71028467, -26560054.7945392 ])

In [112]:
linear_rmse_scores = np.sqrt(-linear_scores)
linear_rmse_scores

array([5380.79062703, 6143.95758539, 4624.58897262, 5001.64190144,
       5153.64480679])

In [113]:
display_scores(linear_rmse_scores)

Scores: [5380.79062703 6143.95758539 4624.58897262 5001.64190144 5153.64480679]
Mean: 5260.924778655179
Standard Dev: 505.52440499142983


### DecisionTree

In [114]:
tree_reg = DecisionTreeRegressor(random_state=42)

In [115]:
model_tree_reg2 = Pipeline(steps = [
    ('tranformer', transformer),
    ('tree_reg', tree_reg)
])

In [116]:
model_tree_reg2.fit(X_train, y_train)

Pipeline(steps=[('tranformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('Scaler',
                                                                   StandardScaler())]),
                                                  ['Poblacion', 'PIB',
                                                   'Desocupacion',
                                                   'EficienciaPrimaria',
                                                   'Medicos']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  ['Tipo de delito'])])),
                ('tree_reg', DecisionTreeRegressor(random_state=42))])

In [117]:
print(model_linear_regression2.predict(muestra_datos))
print(model_tree_reg2.predict(muestra_datos))

[ -318.12701476    79.95188995 -1200.27987468  4414.57842882
 -1212.52053693]
[  1. 879.   3. 344.  35.]


In [118]:
list(y_test.iloc[:5])

[42.0, 829.0, 0.0, 532.0, 465.0]

In [119]:
from sklearn.model_selection import cross_val_score

tree_reg_scores = cross_val_score(model_tree_reg2, X, y,
                               scoring = 'neg_mean_squared_error', cv = 5)

tree_reg_scores

array([-8147908.23046875, -2743960.96918403, -2111280.65625   ,
       -1938337.52777778, -5172550.37413194])

In [120]:
tree_reg_scores = np.sqrt(-tree_reg_scores)
tree_reg_scores

array([2854.45410376, 1656.49055813, 1453.02465783, 1392.24190706,
       2274.32415766])

In [121]:
display_scores(tree_reg_scores)

Scores: [2854.45410376 1656.49055813 1453.02465783 1392.24190706 2274.32415766]
Mean: 1926.1070768886389
Standard Dev: 559.3917052674294


### Random Forest

In [122]:
forest_reg = RandomForestRegressor(n_estimators = 10, random_state = 42)

In [123]:
model_forest_reg2 = Pipeline(steps = [
    ('tranformer', transformer),
    ('forest_reg', forest_reg)
])

In [124]:
model_forest_reg2.fit(X_train, y_train)

Pipeline(steps=[('tranformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('Scaler',
                                                                   StandardScaler())]),
                                                  ['Poblacion', 'PIB',
                                                   'Desocupacion',
                                                   'EficienciaPrimaria',
                                                   'Medicos']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  ['Tipo de delito'])])),
                ('forest_reg',
                 RandomForestRegressor(n_estimators=10, random_state=42))])

In [125]:
print(model_linear_regression2.predict(muestra_datos))
print(model_tree_reg2.predict(muestra_datos))
print(model_forest_reg2.predict(muestra_datos))


[ -318.12701476    79.95188995 -1200.27987468  4414.57842882
 -1212.52053693]
[  1. 879.   3. 344.  35.]
[  5.9 781.6   3.9 377.  216.9]


In [126]:
list(y_test.iloc[:5])

[42.0, 829.0, 0.0, 532.0, 465.0]

In [127]:
from sklearn.model_selection import cross_val_score

forest_reg_scores = cross_val_score(model_forest_reg2, X, y,
                               scoring = 'neg_mean_squared_error', cv = 5)

forest_reg_scores

array([-3775157.14891927, -3408703.21687934, -1388713.38810764,
        -972034.68722656, -4358570.39980903])

In [128]:
forest_reg_scores = np.sqrt(-forest_reg_scores)
forest_reg_scores

array([1942.97636345, 1846.26737416, 1178.43684095,  985.918195  ,
       2087.71894656])

In [129]:
display_scores(forest_reg_scores)

Scores: [1942.97636345 1846.26737416 1178.43684095  985.918195   2087.71894656]
Mean: 1608.2635440218346
Standard Dev: 440.59521236470164
